# THESIS_code-sample

This code is part of the TU Delft master's thesis entitled "Solar-Climatic Configuration: a model for feed-forward optimization of building envelopes as to solar energy potential", which was presented in June 2021.

The goal of this code is the generation of an optimal envelope according to a visibility target (e.g. the sky). The visibility of the sky is maximized through a vectorized process after computing several factors. These factors are based on the pre-computation of the interdependencies of all the spatial elements/voxels of the envelope according to the defined visibility target.

Author: Anastasia Florou
Date: June 2021

## 0. Setup

### 0.1 Import libraries

In [28]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import pandas as pd
import resources as ths
import pickle as pk
from sklearn.preprocessing import minmax_scale as sk_minmax

### 0.2 Load meshes

In [29]:
context_path = os.path.relpath("data/context_mesh.obj")
context_mesh = tm.load(context_path)

envelope_path = os.path.relpath("data/envelope_mesh.obj")
envelope_mesh = tm.load(envelope_path)

### 0.3 Check watertightness

In [30]:
# check watertightness
print(context_mesh.is_watertight)
print(envelope_mesh.is_watertight)

False
True


### 0.4 Visualize meshes

In [31]:
p = pv.Plotter(notebook=True)
p.add_mesh(ths.tri_to_pv(context_mesh), color='#aaaaaa')
p.add_mesh(ths.tri_to_pv(envelope_mesh), color='#0dbadc')
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(155.24196096950666, 178.50525340133802, 163.7824467092146),
 (10.109514236450195, 33.372806668281555, 18.649999976158142),
 (0.0, 0.0, 1.0)]

## 1. Voxelize envelope

### 1.1 Voxelization settings

In [32]:
# voxelization settings
vs = 8 # voxel size
unit = [vs, vs, vs]

# initialize the base lattice
base_lattice = tg.lattice(envelope_mesh.bounds, unit=unit, default_value=1, dtype=int)

# returns True(inside the mesh) and False(outside the mesh)
interior_condition = envelope_mesh.contains(base_lattice.centroids) 

# reshape the interior condition to the shape of the base_lattice
interior_array = interior_condition.reshape(base_lattice.shape)

# convert the interior array into a lattice
envelope_lattice = tg.to_lattice(interior_array, base_lattice.minbound, base_lattice.unit)

### 1.2 Visualize voxels

In [33]:
p = pv.Plotter(notebook=True)
envelope_lattice.fast_vis(p)
p.add_mesh(ths.tri_to_pv(context_mesh), color='#aaaaaa')
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(156.9542188055995, 179.20956891851455, 164.84470456914931),
 (10.109514236450195, 32.364864349365234, 18.0),
 (0.0, 0.0, 1.0)]

### 1.3 Save to csv

In [34]:
csv_path = os.path.relpath("data/voxelized_envelope.csv")
envelope_lattice.to_csv(csv_path)

## 2. Reference vectors

### 2.1 Construct, subdivide skydome and extract sky vectors

In [35]:
# create a sphere 
sphere_mesh = tm.creation.icosphere(subdivisions=2, radius= 50.0, color=None)

# extract vertices (vectors) from sphere
sph_vectors = np.copy(sphere_mesh.vertices)

# keep only the vectors with positive Z (points of upper hemisphere)
sky_vectors = sph_vectors[sph_vectors[:,2] > 0.0]

# convert to array
sky_vectors = np.array(sky_vectors)

### 2.2 Visualize sky vectors

In [36]:
p = pv.Plotter(notebook=True)
envelope_lattice.fast_vis(p)
p.add_mesh(ths.tri_to_pv(context_mesh), color='#aaaaaa')
p.add_points(sky_vectors * 10, color='#DDD53F')
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(1610.0759513385235, 1610.0759513385235, 1858.0759513385235),
 (0.0, 0.0, 248.0),
 (0.0, 0.0, 1.0)]

### 2.3 Save computed vectors

In [37]:
# save sky vectors array
pk.dump(sky_vectors, open("data/skyvectors.pk", "wb"))

## 3. Compute interdependencies

Construct a graph of intervisibilities (G) given a set of voxels and reference vectors.

$G$: Directed multigraph of visibility dependency of voxels regarding a particular visibility target represented as a tensor (a stack of matrices), whose dimensions respectively correspond to (obscuring voxels, obscured voxel, vision rays).

i.e. $G_{i,j,k}$ will be 1 if a voxel $v_i$ obscures a voxel $v_j$ for receiving a ray $r_k$, and 0 otherwise

### 3.1 Prepare computation

In [38]:
# load reference vectors
ref_vectors = (pk.load(open("data/skyvectors.pk", "rb"))) 

# construct reference directions array
ref_dir_array = np.array(ref_vectors)

# extract voxel centroids
vox_cts = envelope_lattice.centroids

# create mesh cuboids in the position of voxels
_, cuboids = ths.cuboids_from_voxels(envelope_lattice)

### 3.2 Rays sources and directions

In [39]:
# shoot from all the voxels to all the reference directions
ray_dir = np.tile(ref_dir_array, [len(vox_cts),1])
ray_src = np.tile(vox_cts, [1, len(ref_dir_array)]).reshape(-1, 3)

### 3.3 Compute intersections

In [40]:
# intersection of rays from voxel centroids to visibility target with all voxel faces
hitface_id, ray_id = cuboids.ray.intersects_id(ray_origins=ray_src, ray_directions=ray_dir, multiple_hits=True)

### 3.4 Construct interdependencies graph

In [41]:
G = ths.construct_graph(ref_vectors, hitface_id, ray_id, envelope_lattice, 12)

### 3.5 Save graph

In [42]:
# save intervisibilities graph
pk.dump(G, open("data/G_graph_sky.pk", "wb"))

## 4. Contextual shading

Construct a matrix of visibility of rays (U) given a set of voxels, a context mesh and an array of reference vectors.

$U$: a matrix representing the visibility of unobstructed $r_k$ for $v_i$, given a partially obstructing context, whose entries indicate if $v_i$ receives a ray $r_k$

### 4.1 Prepare computation

In [43]:
# construct reference directions array
ref_dir_array = np.array(ref_vectors)

# voxel centroids
vox_cts = envelope_lattice.centroids

# shoot from all the voxels to all the reference directions
ray_dir = np.tile(ref_dir_array, [len(vox_cts),1])
ray_src = np.tile(vox_cts, [1, len(ref_dir_array)]).reshape(-1, 3)

### 4.2 Compute intersections & Construct graph

In [44]:
U = ths.construct_visiblerays(ray_src, ray_dir, context_mesh, envelope_lattice)

### 4.3 Save graph

In [45]:
# save matrix
pk.dump(U, open("data/U_graph_sky.pk", "wb"))

## 5. Cost index evaluation

The  cost index is a numerical value that shows how "costly" or "annoying" is every voxel for a configuration.

### 5.1 Cost index calculation

The formula to calculate this cost is based on the computation of two factors. First one is the Obscuring index, which expresses the visibility potential that one voxel prevents from the others:
$$ Obscuring\_index = [\mathbf{G}_{k,i,j}^T]_{m \times n \times n}[\mathbf{x}_i]_{n \times 1} $$

The second factor is the Obscured index, which expresses the visibility potential that is denied from a voxel because of the other voxels:

$$ Obscured\_index = [\mathbf{G}_{k,j,i}^T]_{m \times n \times n}[\mathbf{x}_i]_{n \times 1} $$

A voxel is considered costly for a configuration when it obstructs a great amount of rays from the rest of the voxels while not being much obscured itself. To account for the partially obscuring context, the Hadamard product with the visible rays matrix is performed and finally weights are assigned to the rays.

$$
     C(\mathbf{x}) = \mathbf{w}^T\left([\mathbf{G}_{k,i,j}^T]\mathbf{x} - [\mathbf{G}_{k,j,i}^T]\mathbf{x}\right)\odot \mathbf{U}
$$


In [49]:
# tansparency vector x (the vector that contains the information of which voxels are occupied and which are not)
x = np.ones(len(vox_cts)) # for a full envelope

# equally weighted rays
w = np.ones(ref_vectors.shape[0])

# calculate cost index
cost_index = ths.cost_index_calculation(G, U, x, w)

# normalize values
cost_index_norm = sk_minmax(1 - cost_index)

# reshape and store to lattice
cost_index_lat = ths.reshape_and_store_to_lattice(cost_index_norm, envelope_lattice)

### 5.2 Cost index visualization


In [50]:
p = pv.Plotter(notebook=True)

#choose which lattice to visualize
base_lattice = cost_index_lat

grid = pv.UniformGrid() # Create the spatial reference
grid.dimensions = base_lattice.shape # Set the grid dimensions
grid.origin = base_lattice.minbound # The bottom left corner of the data set
grid.spacing = base_lattice.unit # These are the cell sizes along each axis

# Add the data values to the cell data
grid.point_arrays["Score"] = base_lattice.flatten(order="F")  # Flatten the Lattice

# add sun vectors
p.add_mesh(ref_vectors*10, color="#FFA500")

# adding the meshes
p.add_mesh(ths.tri_to_pv(context_mesh), opacity=0.1, style='wireframe')

# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6])*1.0
p.add_volume(grid, cmap="coolwarm", clim=[0.0, 1.0],opacity=opacity, shade=True)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(1608.5237322042385, 1608.5237322042385, 1858.5237322042385),
 (0.0, 0.0, 250.0),
 (0.0, 0.0, 1.0)]

### 5.3 Envelope selection

In [54]:
p = pv.Plotter(notebook=True)

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
envelope_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):

    lattice = np.copy(base_lattice)
    lattice[base_lattice < value] *= 0.0
    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F")  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([0.001, 1.0])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False, clim=[0.0, 1.0])

    return

p.add_slider_widget(create_mesh, [0.0, 1.0], title='Time', value=0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(93.27406610312548, 93.27406610312548, 93.27406610312548),
 (16.0, 16.0, 16.0),
 (0.0, 0.0, 1.0)]

### 5.4 Apply threshold

In [56]:
threshold = 0.35
new_avail_lattice = cost_index_lat > threshold

p = pv.Plotter(notebook=True)
new_avail_lattice.fast_vis(p)
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(93.27406610312548, 93.27406610312548, 93.27406610312548),
 (16.0, 16.0, 16.0),
 (0.0, 0.0, 1.0)]

### 5.5 Save new envelope

In [ ]:
csv_path_new = os.path.relpath('data/new_envelope_lattice.csv')
new_avail_lattice.to_csv(csv_path_new)